# Word Embeddings : le modèle Word2Vec

## Imports

In [43]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

In [44]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [45]:
infile = f"../../data/sents.txt"
sentences = MySentences(infile)

In [ ]:
bigram_phrases = Phrases(sentences)

In [13]:
len(bigram_phrases.vocab.keys())

15850647

### Conversion des `Phrases` en objet `Phraser`


In [16]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

### Extraction des trigrams

In [17]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [18]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [19]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [20]:
print(corpus[:100])

[['mi', 'imnri', 'r', 'i', '<<', 'i', 'i', 'hmu', "'", 'i', '/', 'tx', "-'", 'l', ':', 'marche', 'tenu', 'hors', 'villa', ',', 'la', '9', '.'], ['--', 'u', 'a', 'ete', 'vaain', 'si', 'teicj', '>>', 'm', 'races_indigenes', 'de', 'fr', '.'], ['31', '<)', 'a', '5s', "'", 'k', '131', 'de', '.'], ['rasa', 'iichakdui', "'", 'te', ',', 'do', '(', 'r', '.', '3s0', 'h', '710', '.', 'taureaux', 'iallsenas', ',>', 'ia', 'u', '\\', '--', 'a', '--', ';', '0ii', '.'], ['hollandais', ',', 'dufr', '.'], ['0', '.'], ['--', 'a', '9', '.--', 'la', 'idto', '-', 'vachei', 'laitieres', ':', 'bn', 'vante', '1q', '.'], ['vendues', '3', '\\', 'au', 'prix', 'la', '410', 'a', '*', '<<', 'i', 'h', '\\;', 'genisses', ',', 'kl', '.'], ["'.", '9', '.'], ['i', 'l', '.', '2', 'i', '.', 'id', '.'], ['da', '370', 'i', '6lutr', '.'], ['marche', 'a', '<', 'u', 'porcs', '.'], ['--', 'categorie', 'de', 'lt', 'ilashtya', ':', "'", '237', 'on', 'vente', ';', 'vendus', '1', 'm', '.', 'do', "'", '2', 'i', '.--', 'a', ';:,', 'l'

## Entrainement d'un modèle Word2Vec sur ce corpus

In [21]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 23min 43s
Wall time: 20min 55s


### Sauver le modèle dans un fichier

In [23]:
outfile = f"../../data/newspapers.model"
model.save(outfile)

### Charger le modèle en mémoire

In [49]:
model = Word2Vec.load("../../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [50]:
model.wv["depute"]

array([-1.6095663 , -0.29488546,  3.6397498 ,  0.5866914 , -3.7378879 ,
       -2.2834668 ,  3.5434563 , -0.4665857 , -3.994869  , -2.1728642 ,
        1.4388734 , -5.4051313 ,  0.08644336,  1.1615486 ,  5.1337767 ,
       -1.5620943 , -0.7576948 , -4.3216257 ,  0.34956396, -2.0865264 ,
       -5.1336007 ,  0.69817334, -1.625669  , -3.3933058 , -1.3503561 ,
        0.4768005 , -2.7889268 , -3.8527367 ,  1.3063163 , -1.3188568 ,
       -7.248159  , -5.710964  ], dtype=float32)

### Calculer la similarité entre deux termes

In [51]:
model.wv.similarity("ministre", "roi")

0.69407684

### Chercher les mots les plus proches d'un terme donné

In [58]:
model.wv.most_similar("royaume", topn=10)

[('protectorat', 0.8697417974472046),
 ('rattachement', 0.8495016098022461),
 ('pays', 0.8492189049720764),
 ('pacte_atlantique', 0.846608579158783),
 ('monde_occidental', 0.8425431251525879),
 ('morcellement', 0.8339406847953796),
 ('cameroun', 0.8307098150253296),
 ('pouvoir_executif', 0.8299675583839417),
 ('nouvel_etat', 0.8291213512420654),
 ('thibet', 0.8272477984428406)]

### Faire des recherches complexes à travers l'espace vectoriel

In [62]:
print(model.wv.most_similar(positive=['lyon', 'paris'], negative=['france']))

[('marseille', 0.7970285415649414), ('strasbourg', 0.781307578086853), ('limoges', 0.7499791979789734), ('boston', 0.745964527130127), ('casablanca', 0.7383399605751038), ('bastia', 0.7364376187324524), ('rouen', 0.7352166175842285), ('san_francisco', 0.7247501015663147), ('chicago', 0.7223731279373169), ('paria', 0.7206785678863525)]
